In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import ot
import cvxpy as cp

# Supplementary Packages
#import scipy.stats as stats
#import seaborn as sns
#import scipy.special as sps
import time as t

## Functions

In [2]:
def baryc_proj(source, target, method):
    
    n1 = source.shape[0]
    n2 = target.shape[0]   
    p = source.shape[1]
    a_ones, b_ones = np.ones((n1,)) / n1, np.ones((n2,)) / n2
    
    M = ot.dist(source, target)
    M = M.astype('float64')
    M /= M.max()
    
    if method == 'emd':
        OTplan = ot.emd(a_ones, b_ones, M, numItermax = 1e7)
        
    elif method == 'entropic':
        OTplan = ot.bregman.sinkhorn_stabilized(a_ones, b_ones, M, reg = 5*1e-3)
    
    # initialization
    OTmap = np.empty((0, p))

    for i in range(n1):
        
        # normalization
        OTplan[i,:] = OTplan[i,:] / sum(OTplan[i,:])
    
        # conditional expectation
        OTmap = np.vstack([OTmap, (target.T @ OTplan[i,:])])
    
    OTmap = np.array(OTmap)
    
    return(OTmap)

In [3]:
def DSCreplicationV2(target, controls, method = 'emd'):
    
    n = target.shape[0]
    d = target.shape[1]
    J = len(controls)
    
    
    # Barycentric Projection
    G_list = []
    proj_list = []
    for i in range(len(controls)):
        temp = baryc_proj(target, controls[i], method)
        G_list.append(temp)
        proj_list.append(temp - target)
    
    
    # Obtain optimal weights
    mylambda = cp.Variable(J)

    objective = cp.Minimize(
                    cp.sum(
                    cp.sum(
                    cp.sum([a*b for a,b in zip(mylambda, proj_list)])**2))/n
                    )
    
    constraints = [mylambda >= 0, mylambda <= 1, cp.sum(mylambda) == 1]

    prob = cp.Problem(objective, constraints)
    prob.solve()
    
    weights = mylambda.value
    replication = sum([a*b for a,b in zip(weights, G_list)])
    
    
    return(weights, replication)

## Medicare Data

In [4]:
def read_medicaid(file_name, sample = False):
    
    columns = ['HINSCAID','EMPSTAT','UHRSWORK','INCWAGE']
    df = pd.read_csv(file_name)[columns]
    
    if sample:
        df = df.sample(5000, random_state = 31)
    
    return(np.array(df))

In [5]:
import os, glob
medidata = []

# check path
for file in glob.glob("workingData/medicaid/*.csv"):
    medidata.append(read_medicaid(file, sample = True))

medidata.insert(0, medidata.pop(12)) # Move Montana to front of list(why is mt 12?)
medi_target = medidata[0]
medi_controls = medidata[1:]

### Test Run

In [ ]:
ts = t.time()

weightsm, replicationm = DSCreplicationV2(medi_target, medi_controls)

te = t.time() - ts
# round integer columns
replicationm[:,0:2] = replicationm[:,0:2].round(decimals = 0).astype('int64')

In [ ]:
medi_target[0:5,:]

In [ ]:
replicationm[0:5,:]

In [ ]:
weightsm

In [ ]:
sum(sum((medi_target - replicationm)**2))

In [ ]:
te

## RESULTS

### HINSCAID

In [ ]:
5000 - sum(medi_target[:,0] == replicationm[:,0]) ## number of misses

### EMPSTAT

In [ ]:
5000 - sum(medi_target[:,1] == replicationm[:,1]) ## number of misses

### UHRSWORK

In [ ]:
plt.hist(medi_target[:,2])

In [ ]:
plt.hist(replicationm[:,2])

### INCWAGE

In [ ]:
plt.hist(medi_target[:,3])

In [ ]:
plt.hist(replicationm[:,3])

### Notes:
* This replication was obtained using all 12 controls, all of them sampled without replacement at n=1500.
* I think the replications are quite accurate; they seem to 'preserve the geometry' of the target distributions
* The right tail of the INCWAGE column is not replicated as well. However, the point at the right end are outliers and I feel like all machine learning are prone to this level of error.
* One thing to figure out is when to do the rounding. For this test run, binary columns are rounded after the replication, but I am unsure of the theoretical validity. One option is to round at the barycentric projection step, but in this case, we might have to round once more after we multiply the optimal weights.
* This test run used 1500 observations, and it took 4 seconds to run on my personal laptop. If we were to utilize the GL cluster, I believe we can use even more than 10k observations. And another test run I did with 150 was worse than with 1500, so we might have even better replications with 10k.